<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%202/Modulo_2_Ejercicio_6_Pipeline_Datos_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Ejercicio 6: Implementación de un Pipeline Completo de Preparación de Datos**
## Automatización del flujo de limpieza, transformación y validación de datos.



## Introducción

En este ejercicio, construiremos un **pipeline completo de preparación de datos**, integrando todas las fases necesarias para limpiar, transformar y validar datos industriales antes de su análisis o modelado.

El preprocesamiento de datos es un paso crítico en cualquier proyecto de análisis, ya que asegura que la información utilizada sea de alta calidad, sin valores atípicos, sin duplicados y correctamente normalizada.

### Objetivos del ejercicio:
- Diseñar una función para cargar datos en distintos formatos (**CSV, JSON**).
- Implementar un **pipeline de preprocesamiento** que incluya:
  - Eliminación de **duplicados**.
  - Manejo de **valores nulos** mediante imputación.
  - **Normalización** de datos numéricos.
  - Conversión de **variables categóricas**.
- Crear **reglas de validación** para garantizar la calidad de los datos.
- Guardar el dataset final en distintos formatos listos para su uso posterior.
- Evaluar la **robustez del pipeline** con distintos datasets.

### Conceptos clave:
- **Automatización del preprocesamiento**  
- **Validación de datos**  
- **Implementación de flujos de datos eficientes**  


In [1]:

# Celda 1: Importación de librerías necesarias

import pandas as pd
import numpy as np
import json

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Configuración de opciones de visualización
pd.set_option("display.max_columns", None)

print("Librerías importadas correctamente.")


Librerías importadas correctamente.



## Descripción del Dataset

Para este ejercicio, utilizaremos el dataset **"Control de calidad en empaques de alimentos"**, alojado en GitHub.  
Este dataset contiene información sobre la calidad de empaques utilizados en la industria alimentaria.

### **Fuente del dataset:**  
- **CSV:** [Control de calidad en empaques de alimentos](https://raw.githubusercontent.com/dtoralg/IE_Calidad_ML/refs/heads/main/Data/control_calidad_empaques.csv)

### **Estructura del dataset:**
| Columna               | Descripción |
|-----------------------|-------------|
| ID_envase            | Identificador único del envase |
| Peso_envase          | Peso del envase en gramos |
| Espesor_material     | Espesor del material en mm |
| Dureza_superficie    | Dureza de la superficie en N/mm² |
| Temperatura_sellado  | Temperatura de sellado en °C |
| Tiempo_prensado      | Tiempo de prensado en segundos |
| Proveedor_material   | Nombre del proveedor (categórico) |
| Tipo_envase          | Tipo de envase (categórico) |
| Color_material       | Color del material (categórico) |
| Defecto_detectado    | Tipo de defecto presente (multiclase) |


In [2]:

# Celda 2: Cargar el dataset desde GitHub

url_csv = "https://raw.githubusercontent.com/dtoralg/IE_Calidad_ML/refs/heads/main/Data/control_calidad_empaques.csv"
df = pd.read_csv(url_csv)

# Mostrar las primeras filas del dataset
df.head()


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Proveedor_material,Tipo_envase,Color_material,Defecto_detectado
0,1,52.483571,0.251530,131.236814,166.001783,8.035426,Proveedor_7,Envase_4,Azul,Rasgado
1,2,49.308678,0.142232,98.115434,185.478228,4.851902,Proveedor_4,Envase_1,Transparente,Rasgado
2,3,53.238443,0.228772,141.912131,178.792377,4.524469,Proveedor_6,Envase_1,Azul,Rasgado
3,4,57.615149,0.169038,72.227237,177.167362,5.515039,Proveedor_4,Envase_2,Blanco,Doble capa
4,5,48.829233,0.183630,93.146984,187.675692,4.936259,Proveedor_9,Envase_4,Azul,Sin defecto


In [3]:

# Celda 3: Definir una función para cargar datos en distintos formatos

def cargar_datos(ruta, formato="csv"):
    """Carga datos en formato CSV o JSON."""
    if formato == "csv":
        return pd.read_csv(ruta)
    elif formato == "json":
        return pd.read_json(ruta)
    else:
        raise ValueError("Formato no soportado. Usa 'csv' o 'json'.")

# Cargar el dataset desde CSV usando la función
df = cargar_datos(url_csv, formato="csv")
df.head()


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Proveedor_material,Tipo_envase,Color_material,Defecto_detectado
0,1,52.483571,0.251530,131.236814,166.001783,8.035426,Proveedor_7,Envase_4,Azul,Rasgado
1,2,49.308678,0.142232,98.115434,185.478228,4.851902,Proveedor_4,Envase_1,Transparente,Rasgado
2,3,53.238443,0.228772,141.912131,178.792377,4.524469,Proveedor_6,Envase_1,Azul,Rasgado
3,4,57.615149,0.169038,72.227237,177.167362,5.515039,Proveedor_4,Envase_2,Blanco,Doble capa
4,5,48.829233,0.183630,93.146984,187.675692,4.936259,Proveedor_9,Envase_4,Azul,Sin defecto


In [4]:

# Celda 4: Implementación del pipeline de preprocesamiento

def preprocesar_datos(df):
    """Realiza limpieza, normalización y transformación de datos."""

    # Eliminar duplicados
    df = df.drop_duplicates()

    # Manejo de valores nulos
    imputer = SimpleImputer(strategy="median")
    columnas_numericas = df.select_dtypes(include=[np.number]).columns
    df[columnas_numericas] = imputer.fit_transform(df[columnas_numericas])

    # Normalización de datos numéricos
    scaler = StandardScaler()
    df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

    # Conversión de variables categóricas con One-Hot Encoding
    df = pd.get_dummies(df, columns=["Proveedor_material", "Tipo_envase", "Color_material"], drop_first=True)

    return df

# Aplicar el pipeline al dataset
df_procesado = preprocesar_datos(df)

print("Preprocesamiento completado.")
df_procesado.head()


Preprocesamiento completado.


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Defecto_detectado,Proveedor_material_Proveedor_10,Proveedor_material_Proveedor_2,Proveedor_material_Proveedor_3,Proveedor_material_Proveedor_4,Proveedor_material_Proveedor_5,Proveedor_material_Proveedor_6,Proveedor_material_Proveedor_7,Proveedor_material_Proveedor_8,Proveedor_material_Proveedor_9,Tipo_envase_Envase_2,Tipo_envase_Envase_3,Tipo_envase_Envase_4,Tipo_envase_Envase_5,Color_material_Blanco,Color_material_Transparente,Color_material_Verde
0,-1.732033,0.495301,1.030712,0.896623,-1.397472,2.913047,Rasgado,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False
1,-1.731999,-0.139106,-1.157569,-0.181028,0.547082,-0.122091,Rasgado,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
2,-1.731964,0.646140,0.575069,1.243960,-0.120442,-0.434262,Rasgado,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,-1.731930,1.520693,-0.620881,-1.023337,-0.282685,0.510138,Doble capa,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False
4,-1.731895,-0.234909,-0.328734,-0.342683,0.766480,-0.041665,Sin defecto,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False


In [5]:

# Celda 5: Definir reglas de validación de datos

def validar_datos(df):
    """Verifica calidad del dataset preprocesado."""
    if df.isnull().sum().sum() > 0:
        print("Error: Hay valores nulos en el dataset.")
    elif df.duplicated().sum() > 0:
        print("Error: Existen registros duplicados.")
    else:
        print("Validación superada: Los datos están limpios y listos para su uso.")

# Aplicar validación al dataset procesado
validar_datos(df_procesado)


Validación superada: Los datos están limpios y listos para su uso.


In [6]:

# Celda 6: Guardar el dataset final en distintos formatos

df_procesado.to_csv("control_calidad_empaques_procesado.csv", index=False)
df_procesado.to_json("control_calidad_empaques_procesado.json", orient="records")

print("Dataset guardado en formato CSV y JSON.")


Dataset guardado en formato CSV y JSON.


In [7]:

# Celda 7: Evaluar la robustez del pipeline con un dataset de prueba

# Crear un dataset de prueba con valores faltantes y categorías nuevas
df_test = df.copy()
df_test.loc[0, "Peso_envase"] = np.nan  # Introducir un valor nulo
df_test.loc[1, "Proveedor_material"] = "Proveedor_X"  # Categoría nueva

# Aplicar el pipeline al dataset de prueba
df_test_procesado = preprocesar_datos(df_test)
validar_datos(df_test_procesado)

df_test_procesado.head()


Validación superada: Los datos están limpios y listos para su uso.


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Defecto_detectado,Proveedor_material_Proveedor_10,Proveedor_material_Proveedor_2,Proveedor_material_Proveedor_3,Proveedor_material_Proveedor_4,Proveedor_material_Proveedor_5,Proveedor_material_Proveedor_6,Proveedor_material_Proveedor_7,Proveedor_material_Proveedor_8,Proveedor_material_Proveedor_9,Proveedor_material_Proveedor_X,Tipo_envase_Envase_2,Tipo_envase_Envase_3,Tipo_envase_Envase_4,Tipo_envase_Envase_5,Color_material_Blanco,Color_material_Transparente,Color_material_Verde
0,-1.732033,0.001659,1.030712,0.896623,-1.397472,2.913047,Rasgado,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False
1,-1.731999,-0.139101,-1.157569,-0.181028,0.547082,-0.122091,Rasgado,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False
2,-1.731964,0.646145,0.575069,1.243960,-0.120442,-0.434262,Rasgado,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,-1.731930,1.520700,-0.620881,-1.023337,-0.282685,0.510138,Doble capa,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False
4,-1.731895,-0.234904,-0.328734,-0.342683,0.766480,-0.041665,Sin defecto,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False



## Conclusiones

En este ejercicio hemos construido un **pipeline completo de preprocesamiento de datos**, integrando todas las fases necesarias para preparar datos industriales.

### Puntos clave:
- Se diseñó una **función modular** para cargar datos en distintos formatos.
- Se implementó un **pipeline de limpieza**, que incluye eliminación de duplicados y manejo de valores nulos.
- Se aplicó **normalización** a las variables numéricas para estandarizar el dataset.
- Se convirtió información categórica en variables numéricas con **One-Hot Encoding**.
- Se establecieron **reglas de validación** para asegurar la calidad del dataset.
- Se evaluó la **robustez del pipeline** con datos de prueba.

### Posibles mejoras:
- Adaptar el pipeline para incluir validaciones más complejas.
- Implementar técnicas avanzadas de imputación de valores nulos.
- Optimizar la transformación de variables categóricas en datasets con alta cardinalidad.
